<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/Shopping_vscode_branch/HW05_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

part 1 load file (finish)
part 2 clean data (finish on 12/5)
part 3 tokenize : sentencepiece (finish on 12/6)
part 4 build model encoder decoder (finish on 12/7)
part 5 beam search (finish on 12/8)
part 6 label smooth (finish on 12/9)

part 7 test (finish on 12/10)


In [ ]:
# !pip install 'torch>=1.6.0' editdistance matplotlib sacrebleu sacremoses sentencepiece tqdm wandb
# !pip install --upgrade jupyter ipywidgets
# !git clone https://github.com/pytorch/fairseq.git
# !cd fairseq && git checkout 9a1c497
# !pip install --upgrade ./fairseq/

In [ ]:
# import sys
# import pdb
# import pprint
# import logging
# import os
# import random

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils import data
# import numpy as np
# import tqdm.auto as tqdm
# from pathlib import Path
# from argparse import Namespace
# #from fairseq import utils

# import matplotlib.pyplot as plt

In [ ]:
# seed = 1
# random.seed(seed)
# torch.manual_seed(seed)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
# np.random.seed(seed)
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
shutil.copyfile('/content/drive/MyDrive/ted2020.tgz','/content/ted2020.tgz')

Mounted at /content/drive


'/content/ted2020.tgz'

In [2]:
import tarfile

rawdata_file = "/content/ted2020.tgz"
unzip_path = "/content/train_dev/"
# open file
file = tarfile.open(rawdata_file)

# extracting file
file.extractall(unzip_path)

file.close()


In [3]:
src_lang = "raw.en"
tgt_lang = "raw.zh"

src_path = f"{unzip_path}{src_lang}"
tgt_path = f"{unzip_path}{tgt_lang}"
print(tgt_path)

/content/train_dev/raw.zh


In [4]:
for path in [src_path,tgt_path]:
  with open(path, "r") as f:
  #  data = f.readlines()
  #  print(type(data),data[0:5])#<class 'list'> ['Thank you so much, Chris.\n', "And it'......on.\n']
    data = f.read().splitlines()
    print(type(data),data[0:5],len(data))

<class 'list'> ['Thank you so much, Chris.', "And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.", 'I have been blown away by this conference, and I want to thank all of you for the many nice comments about what I had to say the other night.', 'And I say that sincerely, partly because  I need that.', 'Put yourselves in my position.'] 394066
<class 'list'> ['非常謝謝你，克里斯。能有這個機會第二度踏上這個演講台', '真是一大榮幸。我非常感激。', '這個研討會給我留下了極為深刻的印象，我想感謝大家 對我之前演講的好評。', '我是由衷的想這麼說，有部份原因是因為 —— 我真的有需要!', '請你們設身處地為我想一想！'] 394066


In [ ]:
import re
print(ord("，"))
print(chr(65292))

65292
，


In [5]:
import unicodedata
def to_halfwidth(string):
  return "".join(unicodedata.normalize('NFKC',letter) for letter in string)
print(to_halfwidth("ＡＳＤＦＦＦＦＱ"))

ASDFFFFQ


In [6]:
import string
import re
def clean_s_zh(s):
    s = to_halfwidth(s)
    # step 1 : delete — _
    delete = " _()[]"
    delete_rules = s.maketrans("","",delete)
    s = s.translate(delete_rules)

    # step 2 : replace “” with ""
    to_be_replace = '“”'
    replace = '""'
    replace_dict = dict(zip(to_be_replace,replace))
    # print(replace_dict.items()) : dict_items([('“', '"'), ('”', '"')])

    # step 3 : add **END** before and after punctuation

    """
    The number of sentences in one line may be different
    in line pairs of source and target set.
    so I try to use "。!?" or ".!?" to split sentences.

    """

#    punctuation = "。,;!?()\"~「」"
    punctuation = "。!?"
    for char in punctuation:
      replace_dict[char] = char + "**END**"

    replace_rules = s.maketrans(replace_dict)
    s = s.translate(replace_rules)

    zh_list = s.strip("\n").split("\n")

    return zh_list

def clean_s_en(s):
    s = to_halfwidth(s)

    replace_dict = {}

    delete = "-()[]"
    for char in delete:
      replace_dict[char] = ""

    punctuation = "!?"
    for char in punctuation:
      replace_dict[char] = char + "**END**"
    replace_rules = s.maketrans(replace_dict)
    s = s.translate(replace_rules)

    pattern = re.compile(r"(?<!([.\s\r\n\f][a-zA-Z]))[.]")
    s = pattern.sub("**END**",s)

    en_list = s.strip("\n").split("\n")

    return en_list

In [7]:
pattern = re.compile(r"(?<!([.\s\r\n\f][a-zA-Z]))[.]")
result = pattern.sub("**END**","There are many people in U.S. w.r.t. in Taiwan.Thank you.")
print(result)

There are many people in U.S. w.r.t. in Taiwan**END**Thank you**END**


In [8]:
def load_file(path,fuction):
  with open(path, "r") as f:
    data = f.read()
    return(fuction(data))
src_list = load_file(src_path,clean_s_en)
tgt_list = load_file(tgt_path,clean_s_zh)

In [9]:
with open(tgt_path, "r") as f:
    data = f.readlines()
    print(data[17])
print(tgt_list[17])

我們下了交流道後就開始找餐廳 — 我們發現了一家 Shoney' s餐館。

我們下了交流道後就開始找餐廳—我們發現了一家Shoney's餐館。**END**


In [ ]:
    # """
    # Please ensure that space char " " has bee remove in previous step ,
    # it's the reason s must be translated twice.

    # """



    # print(len(replace_dict.items())) : 13
#    punctuation = "(.,;!?()\"])"


In [10]:
def divide_by_END(s):
    return(s.strip("**END**").split("**END**"))

def devide_en_again(s,punctuation = ":;"):
    replace_dict = {}
    for char in punctuation:
      replace_dict[char] = char + "**END**"

    replace_rules_src = s.maketrans(replace_dict)
    new_s = divide_by_END(s.translate(replace_rules_src))
    return new_s

In [11]:
def check_data_pairs(src_list,tgt_list):
    index = 0
    new_src_list = []
    new_tgt_list = []

    same = 0
    add_next = 0
    split_again = 0
    not_use = 0

    while(index < len(src_list)):

      src = divide_by_END(src_list[index])
      tgt = divide_by_END(tgt_list[index])
      # case 1 : src is as long as tgt , finished.
      if len(src) == len(tgt):
        new_src_list += src
        new_tgt_list += tgt
        same += 1
        index += 1

      else :
        # if it is not the last one : both src and tgt add next sentence
        if index != len(src_list)-1:
          src_add_next = divide_by_END(src_list[index] + src_list[index+1])
          tgt_add_next = divide_by_END(tgt_list[index] + tgt_list[index+1])
          # case 2 : src_add_next is as long as tgt_add_next , finished.
          if len(src_add_next) == len(tgt_add_next):
            new_src_list += src_add_next
            new_tgt_list += tgt_add_next
            add_next += 2
            index += 2

          # using new punctuation to divide tgt (english) sentence.
          else :
            src_add_next = devide_en_again(src_list[index] + src_list[index+1])
            # case 3 : src_add_next is as long as tgt_add_next , finished.
            if len(src_add_next) == len(tgt_add_next):
              new_src_list += src_add_next
              new_tgt_list += tgt_add_next
              split_again +=2
              index += 2

            # case 4 : sentence will not be used.
            else :
              not_use += 1
              # if to_do == 1 :
              #   print(index,src_add_next,tgt_add_next,len(src_add_next),len(tgt_add_next))
              index += 1

        # if it is the last one
        else :
          not_use += 1
          index += 1

    print(index,same,add_next,split_again,not_use)

    return(new_src_list,new_tgt_list)
dataset = check_data_pairs(src_list,tgt_list)
print(dataset[0][0],dataset[1][0])


394066 350481 12188 2972 28425
Thank you so much, Chris 非常謝謝你,克里斯。


In [12]:
data_path = "/content/train_dev/data.txt"
label_path = "/content/train_dev/label.txt"
with open(data_path, "w") as f:
  f.write("\n".join(dataset[0]))
with open(label_path, "w") as f:
  f.write("\n".join(dataset[1]))

In [13]:
with open(data_path, "r") as f:
    data = f.readlines()
    print(data[17])
with open(label_path, "r") as f:
    data = f.readlines()
    print(data[17])

We got off the exit, we found a Shoney's restaurant

我們下了交流道後就開始找餐廳—我們發現了一家Shoney's餐館。



In [14]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00


In [15]:
import sentencepiece as spm
data_info = {
    "data":{"path":data_path,"lang":"en"},
    "label":{"path":label_path,"lang":"zh"},
}
vocab_size = 8000
def tokenized(data_or_label,data_info=data_info):
  spm.SentencePieceTrainer.train(
      input=data_info[data_or_label]["path"],
      model_prefix=f'spm{vocab_size}_{data_info[data_or_label]["lang"]}',
      vocab_size=vocab_size,
      character_coverage=1,
      model_type='unigram', # 'bpe' works as well
      input_sentence_size=1e6,
      shuffle_input_sentence=True,
      normalization_rule_name='nmt_nfkc_cf',
  )

In [ ]:
data_info["data"]["path"]

'/content/train_dev/data.txt'

In [16]:
tokenized("data")
tokenized("label")

In [ ]:

# input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --model_type=%s --character_coverage=%s ' \
#                      '--pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 '
# cmd = input_argument % ("A", "B", 8000, "C", 1)
# print(cmd)
#spm.SentencePieceTrainer(cmd)

In [64]:
import sentencepiece as spm
import torch.utils.data as data
def data_set_preparing():
    src_set = []
    tgt_set = []
    s_en = spm.SentencePieceProcessor(model_file="/content/spm8000_en.model")
    s_zh = spm.SentencePieceProcessor(model_file="/content/spm8000_zh.model")
    with open("/content/train_dev/data.txt","r") as in_f :
      for line in in_f:
        src_set.append(line)
    with open("/content/train_dev/label.txt","r") as in_f :
      for line in in_f:
        tgt_set.append(line)
    train_set, valid_set = data.random_split(list(zip(src_set,tgt_set)),[0.9,0.1])
    # print(" ".join(str(x) for x in s_en.encode(train_set[0][0], out_type=int))+"\n")

    with open("/content/train_dev/tokenized_train_data.txt", 'w') as out_f:
      for line_pair in train_set:
        out_f.write(" ".join(str(x) for x in s_en.encode(line_pair[0], out_type=int))+"\n")
    with open("/content/train_dev/tokenized_valid_data.txt", 'w') as out_f:
      for line_pair in valid_set:
        out_f.write(" ".join(str(x) for x in s_en.encode(line_pair[0], out_type=int))+"\n")
    with open("/content/train_dev/tokenized_train_label.txt", 'w') as out_f:
      for line_pair in train_set:
        out_f.write(" ".join(str(x) for x in s_zh.encode(line_pair[1], out_type=int))+"\n")
    with open("/content/train_dev/tokenized_valid_label.txt", 'w') as out_f:
      for line_pair in valid_set:
        out_f.write(" ".join(str(x) for x in s_zh.encode(line_pair[1], out_type=int))+"\n")

In [65]:
data_set_preparing()

25 397 3 97 240 14 210 35



In [ ]:
spm_model = spm.SentencePieceProcessor(model_file=f'spm{vocab_size}')
in_tag = {
    'train': 'train.clean',
    'valid': 'valid.clean',
    'test': 'test.raw.clean',
}

for split in ['train', 'valid', 'test']:
    for lang in [src_lang, tgt_lang]:
        out_path = prefix/f'{split}.{lang}'

            with open(prefix/f'{split}.{lang}', 'w') as out_f:
                with open(prefix/f'{in_tag[split]}.{lang}', 'r') as in_f:
                    for line in in_f:
                        line = line.strip()
                        tok = spm_model.encode(line, out_type=str)
                        print(' '.join(tok), file=out_f)

In [ ]:
# !head {data_dir+'/'+dataset_name+'/train.'+src_lang} -n 5
# !head {data_dir+'/'+dataset_name+'/train.'+tgt_lang} -n 5
# binpath = Path('./DATA/data-bin', dataset_name)
# if binpath.exists():
#     print(binpath, "exists, will not overwrite!")
# else:
#     !python -m fairseq_cli.preprocess \
#         --source-lang {src_lang}\
#         --target-lang {tgt_lang}\
#         --trainpref {prefix/'train'}\
#         --validpref {prefix/'valid'}\
#         --testpref {prefix/'test'}\
#         --destdir {binpath}\
#         --joined-dictionary\
#         --workers 2


In [ ]:
# def load_data_iterator(task, split, epoch=1, max_tokens=4000, num_workers=1, cached=True):
#     batch_iterator = task.get_batch_iterator(
#         dataset=task.dataset(split),
#         max_tokens=max_tokens,
#         max_sentences=None,
#         max_positions=utils.resolve_max_positions(
#             task.max_positions(),
#             max_tokens,
#         ),
#         ignore_invalid_inputs=True,
#         seed=seed,
#         num_workers=num_workers,
#         epoch=epoch,
#         disable_iterator_cache=not cached,
#         # Set this to False to speed up. However, if set to False, changing max_tokens beyond
#         # first call of this method has no effect.
#     )
#     return batch_iterator

# demo_epoch_obj = load_data_iterator(task, "valid", epoch=1, max_tokens=20, num_workers=1, cached=False)
# demo_iter = demo_epoch_obj.next_epoch_itr(shuffle=True)
# sample = next(demo_iter)
# sample

In [ ]:
# batch = {
#   "id": id, # 每個 example 的 id
#   "nsentences": len(samples), # batch size 句子數
#   "ntokens": ntokens, # batch size 字數
#   "net_input": {
#       "src_tokens": src_tokens, # 來源語言的序列
#       "src_lengths": src_lengths, # 每句話沒有 pad 過的長度
#       "prev_output_tokens": prev_output_tokens, # 上面提到右 shift 一格後的目標序列
#   },
#   "target": target, # 目標序列
# }

In [ ]:
# cuda_env = utils.CudaEnvironment()
# utils.CudaEnvironment.pretty_print_cuda_env_list([cuda_env])
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# # # HINT: transformer 架構
# # from fairseq.models.transformer import (
# #     TransformerEncoder,
# #     TransformerDecoder,
# # )

# def build_model(args, task):
#     """ 按照參數設定建置模型 """
#     src_dict, tgt_dict = task.source_dictionary, task.target_dictionary

#     # 詞嵌入
#     encoder_embed_tokens = nn.Embedding(len(src_dict), args.encoder_embed_dim, src_dict.pad())
#     decoder_embed_tokens = nn.Embedding(len(tgt_dict), args.decoder_embed_dim, tgt_dict.pad())

#     # 編碼器與解碼器
#     # TODO: 替換成 TransformerEncoder 和 TransformerDecoder
#     encoder = RNNEncoder(args, src_dict, encoder_embed_tokens)
#     decoder = RNNDecoder(args, tgt_dict, decoder_embed_tokens)

#     # 序列到序列模型
#     model = Seq2Seq(args, encoder, decoder)

#     # 序列到序列模型的初始化很重要 需要特別處理
#     def init_params(module):
#         from fairseq.modules import MultiheadAttention
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=0.02)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         if isinstance(module, nn.Embedding):
#             module.weight.data.normal_(mean=0.0, std=0.02)
#             if module.padding_idx is not None:
#                 module.weight.data[module.padding_idx].zero_()
#         if isinstance(module, MultiheadAttention):
#             module.q_proj.weight.data.normal_(mean=0.0, std=0.02)
#             module.k_proj.weight.data.normal_(mean=0.0, std=0.02)
#             module.v_proj.weight.data.normal_(mean=0.0, std=0.02)
#         if isinstance(module, nn.RNNBase):
#             for name, param in module.named_parameters():
#                 if "weight" in name or "bias" in name:
#                     param.data.uniform_(-0.1, 0.1)

#     # 初始化模型
#     model.apply(init_params)
#     return model

In [ ]:
# class Seq2Seq(FairseqEncoderDecoderModel):
#     def __init__(self, args, encoder, decoder):
#         super().__init__(encoder, decoder)
#         self.args = args

#     def forward(
#         self,
#         src_tokens,
#         src_lengths,
#         prev_output_tokens,
#         return_all_hiddens: bool = True,
#     ):
#         """
#         Run the forward pass for an encoder-decoder model.
#         """
#         encoder_out = self.encoder(
#             src_tokens, src_lengths=src_lengths, return_all_hiddens=return_all_hiddens
#         )
#         logits, extra = self.decoder(
#             prev_output_tokens,
#             encoder_out=encoder_out,
#             src_lengths=src_lengths,
#             return_all_hiddens=return_all_hiddens,
#         )
#         return logits, extra